<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/imdb/AnalyzeIMDBResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

rm: cannot remove 'abstention': No such file or directory
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-renszbbz/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
Cloning into 'abstention_experiments'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 2948 (delta 3), reused 4 (delta 1), pack-reused 2938
Receiving objects: 100% (2948/2948), 264.98 MiB | 48.38 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [0]:
!ls abstention_experiments/imdb

imdb_abstention_results.json.gz  record_6_model_mgGta_preds
record_10_model_Vs2cs_preds	 record_7_model_Si0xy_preds
record_1_model_FObIR_preds	 record_8_model_yds3D_preds
record_2_model_T4HRC_preds	 record_9_model_dCvxV_preds
record_3_model_ZGX50_preds	 Run_IMDB_Experiments.ipynb
record_4_model_hWBrJ_preds	 test_labels.txt.gz
record_5_model_mBWKc_preds	 valid_labels.txt.gz


In [0]:
import gzip
import json
imdb_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/imdb/imdb_abstention_results.json.gz").read())

In [0]:
imdb_abstention_results.keys()

dict_keys(['metricname_to_abstfrac_to_abstnadapt_to_perf', 'metricname_to_adaptname_to_baselineperfs'])

In [0]:
imdb_abstention_results[
    'metricname_to_abstfrac_to_abstnadapt_to_perf']['auroc']['0.15'].keys()

dict_keys(['auroc', 'tpratfpr_0.01'])

In [0]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np

metricnames = ['auroc', 'tpratfpr_0.01']
abstfracs = ['0.15', '0.3']

abstfrac_to_metricname_to_methodnames = {
    '0.15': {
        'auroc': ['distfrom0.5:None',
                  'distfrom0.5:EM',
                  'js:None',
                  'js:EM',
                  'estwindowdelta_auroc_15pc:None',
                  'estwindowdelta_auroc_15pc:EM',
                  'fumera_auroc_15pc:None',
                  'fumera_auroc_15pc:EM'],
        'tpratfpr_0.01': ['distfrom0.5:None',
                          'distfrom0.5:EM',
                          'js:None',
                          'js:EM',
                          'mcwindowdelta_tpratfpr_15pc:None',
                          'mcwindowdelta_tpratfpr_15pc:EM',
                          'fumera_tpratfpr_15pc:None',
                          'fumera_tpratfpr_15pc:EM']
    },
    '0.3': {
        'auroc': ['distfrom0.5:None',
                  'distfrom0.5:EM',
                  'js:None',
                  'js:EM',
                  'estwindowdelta_auroc_30pc:None',
                  'estwindowdelta_auroc_30pc:EM',
                  'fumera_auroc_30pc:None',
                  'fumera_auroc_30pc:EM'],
        'tpratfpr_0.01': ['distfrom0.5:None',
                          'distfrom0.5:EM',
                          'js:None',
                          'js:EM',
                          'mcwindowdelta_tpratfpr_30pc:None',
                          'mcwindowdelta_tpratfpr_30pc:EM',
                          'fumera_tpratfpr_30pc:None',
                          'fumera_tpratfpr_30pc:EM']
    }   
}

for abstention_results in [imdb_abstention_results]:
  for metricname in metricnames:
    for abstfrac in abstfracs:
      methodnames = abstfrac_to_metricname_to_methodnames[abstfrac][metricname]
      nsamples = len(list(
        abstention_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
                          [metricname][abstfrac].values())[0])
      print(metricname, abstfrac)
      ustats_mat = get_ustats_mat(
          method_to_perfs=
            abstention_results['metricname_to_abstfrac_to_abstnadapt_to_perf']
                [metricname][abstfrac],
          method_names=methodnames,
          max_ustat=nsamples*(nsamples+1)/2 )
      tied_top_methods, tied_worst_methods =(
          get_tied_top_and_worst_methods(
              ustats_mat,
              methodnames,
              #0.05 threshold for one-sided test from normal approx when N=30,
              #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
              threshold=151
          ))
      tied_top_methods = [methodnames[x] for x in tied_top_methods]
      print(tied_top_methods)

auroc 0.15
['estwindowdelta_auroc_15pc:EM']
auroc 0.3
['estwindowdelta_auroc_30pc:EM']
tpratfpr_0.01 0.15
['mcwindowdelta_tpratfpr_15pc:EM']
tpratfpr_0.01 0.3
['mcwindowdelta_tpratfpr_30pc:EM']
